# Connectance & degree distribution

https://poisotlab.github.io/EcologicalNetworks.jl/stable/

In [ ]:
using EcologicalNetworks
using EcologicalNetworksPlots
using Plots
using Random
using Statistics
using Distributions

Random.seed!(42);

## Connectance

In [ ]:
function B(n::Int64, p::Float64)
    return simplify(BipartiteNetwork(rand(Float64, (n,n)) .< p))
end
B(10, 0.1)

In [ ]:
n = 250
P = rand(500)
networks = [B(n,p) for p in P]
histogram(connectance.(networks))

In [ ]:
K(N) = collect(values(degree(N)))

In [ ]:
scatter(connectance.(networks), [var(K(n)) for n in networks])
xaxis!("Connectance")
yaxis!("Variance (degree)")

In [ ]:
scatter(connectance.(networks), [kurtosis(K(n)) for n in networks])
xaxis!("Connectance")
yaxis!("Kurtosis (degree)")

In [ ]:
scatter(connectance.(networks), [skewness(K(n)) for n in networks])
xaxis!("Connectance")
yaxis!("Skewness (degree)")

## Degree

In [ ]:
N = nz_stream_foodweb()[10]

In [ ]:
I = initial(RandomInitialLayout, N)
for step in 1:2000
  position!(ForceDirectedLayout(0.8), I, N)
end
plot(I, N, aspectratio=1, size=(700,700))
scatter!(I, N, nodefill=degree(N), nodesize=degree(N), mc=:YlGnBu)

In [ ]:
k = collect(values(degree(N)));
histogram(k; leg=false, c=:lightgrey)
xaxis!("Degree")

In [ ]:
degrees = collect(1:maximum(k))
p_degrees = zeros(Float64, length(degrees))
for (i, d) in enumerate(degrees)
    p_degrees[i] = sum(k .== d)/richness(N)
end

In [ ]:
scatter(degrees, p_degrees; leg=false)
xaxis!("k")
yaxis!("P(k)")

### Using the Lorenz curve

In [ ]:
function lorenz(N::T) where {T <: AbstractEcologicalNetwork}
    sp = collect(1:richness(N))./richness(N)
    k = sort(collect(values(degree(N))))
    lk = cumsum(k)./(2.0*links(N))
    return sp, lk
end

In [ ]:
N = web_of_life("M_PL_001")
plot(lorenz(N); leg=false, frame=:box, c=:black, aspectratio=1)
xaxis!((0,1), "Fraction of species")
yaxis!((0,1), "Cumulative fraction of interactions")